In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from sklearn.metrics import mean_squared_error

In [2]:
df= pd.read_json("Automotive_5.json",lines = True)

In [3]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B00002243X,"[4, 4]",5,I needed a set of jumper cables for my new car...,"08 17, 2011",A3F73SC1LY51OO,Alan Montgomery,Work Well - Should Have Bought Longer Ones,1313539200
1,B00002243X,"[1, 1]",4,"These long cables work fine for my truck, but ...","09 4, 2011",A20S66SKYXULG2,alphonse,Okay long cables,1315094400
2,B00002243X,"[0, 0]",5,Can't comment much on these since they have no...,"07 25, 2013",A2I8LFSN2IS5EO,Chris,Looks and feels heavy Duty,1374710400
3,B00002243X,"[19, 19]",5,I absolutley love Amazon!!! For the price of ...,"12 21, 2010",A3GT2EWQSO45ZG,DeusEx,Excellent choice for Jumper Cables!!!,1292889600
4,B00002243X,"[0, 0]",5,I purchased the 12' feet long cable set and th...,"07 4, 2012",A3ESWJPAVRPWB4,E. Hernandez,"Excellent, High Quality Starter Cables",1341360000


In [4]:
df = df[['asin','overall','reviewerID']].copy()

In [5]:
df.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [6]:
from sklearn.model_selection import train_test_split
# Splitting train and test data at the start
df_train, df_test = train_test_split(df,test_size=0.2,stratify = df['reviewerID'])

In [7]:
df_train.head()

,asin,overall,reviewerID
20355,B00H72YE7Y,3,A1U8VI6I2MFEU8
19623,B00AX4TWCA,3,AY49JOXTLDOG2
13416,B002J7VX4Y,1,A2ZJ10ZPQEFNQJ
6389,B000EAIERW,5,A2QNW2W6OJMJUX
15041,B003L35EWM,5,A2ZJ6IL2HTS9LO


In [8]:
df_train = df_train.sort_index(axis=0)

In [9]:
df_train.head()

,asin,overall,reviewerID
0,B00002243X,5,A3F73SC1LY51OO
1,B00002243X,4,A20S66SKYXULG2
2,B00002243X,5,A2I8LFSN2IS5EO
3,B00002243X,5,A3GT2EWQSO45ZG
4,B00002243X,5,A3ESWJPAVRPWB4


In [10]:
df_original_train = df_train.copy()
df_train = df_train.pivot(index = "asin" , columns = "reviewerID" , values = "overall")
df_train = df_train.sort_index(axis=1)

In [11]:
df_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00002243Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008BKX5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B00008RW9V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
X = df_train.values
Y = df_train.sum(axis = 1)
Z =df_train.count(axis =1).astype('float64')
Y = Y.div(Z,axis=0)
cosine
df_new_train = df_train.sub(Y,axis=0).fillna(0)

In [13]:
df_new_train.head()

reviewerID,A00473363TJ8YSZ3YAGG9,A0473259F6GQNBD88IYN,A09567722SXKYZTV7OFCJ,A10063PJ5C9WQQ,A100UD67AHFODS,A100WO06OQR8BQ,A1027BOR2MECKT,A105S56ODHGJEK,A108AWE1CYYZVB,A109KHEJL08HW9,...,AZD412AH3W49I,AZFODJ6KW6KQH,AZJPNK73JF3XP,AZOK5STV85FBJ,AZP8DAHJCFWPH,AZPZROZJIKBE8,AZT5UV3ZRB4TT,AZTAK8PFB0NLV,AZV2U6GU5QA6C,AZYBZEVISD0YX
asin,,,,,,,,,,,,,,,,,,,,,
B00002243X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00002243Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008BKX5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B00008RW9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
nbrs_train = NearestNeighbors(n_neighbors= 10 , algorithm='auto',metric ='cosine').fit(df_new_train.values)
distances, indices = nbrs_train.kneighbors(df_new_train,return_distance=True)

In [15]:
distances

array([[0.        , 0.63338631, 0.93545028, ..., 0.95435645, 0.95435645,
        0.96047153],
       [0.        , 0.83606794, 0.84569665, ..., 0.8881966 , 0.90754997,
        0.90871291],
       [0.        , 0.74645372, 0.76095428, ..., 0.951205  , 0.951205  ,
        0.96177646],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.        , 0.73910303, 0.84090906, ..., 0.95600645, 0.96499704,
        0.98625212],
       [0.        , 0.65573481, 0.72603304, ..., 0.92707443, 0.93280632,
        0.93914194]])

In [16]:
indices

array([[   0,   67,  573, ...,  156,  896, 1054],
       [   1, 1089,  396, ...,  663,  109,   71],
       [   2, 1228, 1681, ...,  756,  721,   51],
       ...,
       [1225, 1226, 1221, ..., 1230, 1228, 1218],
       [1833,  169, 1173, ...,  755,  267,  378],
       [1834,  618, 1378, ...,  800,  436, 1679]])

In [17]:
asin_list = df_new_train.index.values.tolist()
index_list = indices.tolist()

In [19]:
for i in range(5):
    string = str(asin_list[i]) + " : "
    for x in index_list[i][1:]:
            string += asin_list[x] + " , "
    print(string)

B00002243X : B0002KRABU , B000EJS9IM , B001D8XA50 , B0027VFZE6 , B000P17NXQ , B0009OMYDS , B00042JSNY , B0018PSASU , B001TE0IMG , 
B00002243Z : B001V8U12M , B000BUQOFO , B000MY3ML8 , B002XL2IBS , B004P91FCO , B000BUOSGG , B000JFJLQA , B0002SRH7O , B0002M9QK6 , 
B00008BKX5 : B002X6VXL4 , B008DREON6 , B003VAYJAS , B000EVYGZA , B000BXKYYI , B0013VT5N8 , B000RN04RO , B000O80B5M , B0002H335A , 
B00008RW9U : B00167Y3ZO , B003UT3S6Q , B0033A5PPO , B00029WT08 , B009OBW28E , B000M8PYO2 , B004HCOE7W , B002NUABUO , B0015PI7A4 , 
B00008RW9V : B000B68V90 , B000CMDPLM , B008FV5R0Y , B0009U7Y2I , B0009IK81E , B003BJNT0K , B0002RNSKK , B0009VIQ1A , B000LQB24G , 


In [20]:
item_avg = df_original_train.groupby(['asin']).mean()
user_avg = df_original_train.groupby(['reviewerID']).mean()
mean_avg = df_original_train["overall"].mean()
item_map = df_new_train.index.values

In [21]:
item_avg.head()

,overall
asin,
B00002243X,4.800000
B00002243Z,4.500000
B00008BKX5,4.500000
B00008RW9U,4.666667
B00008RW9V,4.666667


In [22]:
df_asin = df.groupby('asin').count()
df_asin = df_asin.index.values

In [23]:
df_asin

array(['B00002243X', 'B00002243Z', 'B00008BKX5', ..., 'B00JGM7Z5Q',
       'B00JGP8I1S', 'B00KIAQ8VW'], dtype=object)

In [24]:
df_reviewerID = df.groupby('reviewerID').count()
df_reviewerID = df_reviewerID.index.values

In [25]:
df_reviewerID

array(['A00473363TJ8YSZ3YAGG9', 'A0473259F6GQNBD88IYN',
       'A09567722SXKYZTV7OFCJ', ..., 'AZTAK8PFB0NLV', 'AZV2U6GU5QA6C',
       'AZYBZEVISD0YX'], dtype=object)

In [26]:
df_dict = {}
for line in df_original_train.values:
    if line[0] in df_dict:
        df_dict[line[0]].append(line[2])
    else:
        df_dict[line[0]] = [line[2]]

In [27]:
predict_for_this = []
count = 0
for i in range(len(df_reviewerID)):
    for j in range(len(df_asin)):
#         print(df_asin[i])
        if df_asin[j] not  in  df_dict or not df_reviewerID[i] in df_dict[df_asin[j]]:
            predict_for_this.append((df_reviewerID[i],df_asin[j]))

In [28]:
final_prediction = {}

for i in range(10000):
#__________________________________________________________________________________________________________
# for i in range(len(predict_for_this)): This loop needs to be run instead of above one. 5352407 records: takes time
#__________________________________________________________________________________________________________
    
    item_asin = predict_for_this[i][1]
#     print(item_asin)
    item_userID = predict_for_this[i][0]
#     print(item_asin,item_userID)
    
    predicted_rating = 0
    if item_asin in item_avg.index:
        bi = item_avg.loc[item_asin]['overall'] - mean_avg
    else:
        bi = 0
    if item_userID in user_avg.index:
        bx = user_avg.loc[item_userID]['overall'] - mean_avg
    else:
        bx = 0
            
    baseline_estimate = mean_avg + bi + bx
    predicted_rating = baseline_estimate
#     print(baseline_estimate)
    
    numerator = 0 
    denomenator = 0
    
    if bi != 0 :
        
        item  = np.where(item_map == item_asin)[0][0]
        item_nbrs = indices[item:item+1]
        dist_nbrs = distances[item:item+1]
        cnt = 0;
        k = 0
        
        for j in item_nbrs:
            temp = df_original_train.loc[(df_original_train['asin'] == item_map[j[0]]) &  (df_original_train['reviewerID'] == item_userID)]
            if(temp.size != 0):
                dist = dist_nbrs[cnt:cnt+1]
                numerator += ((1 - dist[0][k])*(temp['overall'].values[0] - (item_avg.loc[temp['asin']]['overall'] - mean_avg) ))

                denomenator += (1 - dist[0][k])
            cnt = cnt + 1
            k+= 1
        if denomenator:
            predicted_rating += numerator/denomenator 

    if item_userID in final_prediction:
        final_prediction[item_userID].append((item_asin,predicted_rating))
    else:
        final_prediction[(item_userID)] = [(item_asin,predicted_rating)]

#___________________________________________________________
#        Sort final_prediction on basis of rating
#___________________________________________________________

In [29]:
for x in final_prediction:
    final_prediction[x] = sorted(final_prediction[x],key = lambda x:x[1],reverse= True)[:10]

In [30]:
final_prediction

{'A00473363TJ8YSZ3YAGG9': [('B0000AXTUY', 4.524545121504457),
  ('B0000AYES4', 4.524545121504457),
  ('B0000UUX34', 4.524545121504457),
  ('B000221M94', 4.524545121504457),
  ('B0002CSQ7A', 4.524545121504457),
  ('B0002JM8PY', 4.524545121504457),
  ('B0002JMV06', 4.524545121504457),
  ('B0002KKTT0', 4.524545121504457),
  ('B0002LBGX2', 4.524545121504457),
  ('B0002NUNMU', 4.524545121504457)],
 'A0473259F6GQNBD88IYN': [('B0000AXTUY', 5.413434010393346),
  ('B0000AYES4', 5.413434010393346),
  ('B0000UUX34', 5.413434010393346),
  ('B000221M94', 5.413434010393346),
  ('B0002CSQ7A', 5.413434010393346),
  ('B0002JM8PY', 5.413434010393346),
  ('B0002JMV06', 5.413434010393346),
  ('B0002KKTT0', 5.413434010393346),
  ('B0002LBGX2', 5.413434010393346),
  ('B0002NUNMU', 5.413434010393346)],
 'A09567722SXKYZTV7OFCJ': [('B0000AXTUY', 5.524545121504457),
  ('B0000AYES4', 5.524545121504457),
  ('B0000UUX34', 5.524545121504457),
  ('B000221M94', 5.524545121504457),
  ('B0002CSQ7A', 5.524545121504457),